In [1]:
#Per iniziare, definisco i dataframe dei libri e dei tipi dei libri.
#DOMANDA: cosa si intende per "Manifestation"?

file = "/data/SMARTDATA/books/BCT_prestiti/items.csv"
DFitems = spark.read.csv(file,header=True)
DFitems.show()

file = "/data/SMARTDATA/books/BCT_prestiti/item_media_types.csv"
DFtypes = spark.read.csv(file,header=True)
DFtypes.show()

#Come posso constatare, è possibile joinare media types e libri tramite l'id media types per vedere a quale genere
#si riferisce il libro e filtrare così i generi non necessari.

+-------+----------------+-------------+---------------+--------------+
|item_id|manifestation_id|item_media_id|home_library_id|inventory_date|
+-------+----------------+-------------+---------------+--------------+
| 288788|          324685|            Q|             27|    2007-06-04|
|1987167|           18421|            S|             13|    2013-01-28|
|2555210|          747441|            F|              2|    2016-09-15|
| 218132|            3490|            F|              2|    2008-03-12|
|1009634|          233431|            F|             19|    2004-05-27|
| 121499|          286272|            F|             27|    2010-09-01|
|2400369|          270193|            F|             16|    2016-01-27|
| 103294|          365773|            F|              2|    2010-06-08|
| 164444|          269506|            F|             20|    2006-05-23|
|2269659|          625603|            A|              3|    2015-02-20|
|1488598|          449444|            T|              2|    2011

In [13]:
#FILTRAGGIO NUMERO 1: GENERI NON NECESSARI
#Per un lettore "medio" credo sia ragionevole filtrare via i materiali multimediali (DVD, VHS, audiolibri, proiezioni
#e altri materiali elettronici), i libri in braille (che rappresentano, immagino, una forte minoranza all'interno del catalogo)
#e i materiali cartografici poiché immagino siano più usati da esperti che da lettori interessati alle raccomandazioni. Discorso
#analogo per le lettere autografe.
#Le microforme (riassunti di libri in versione compatta) credo siano più interessanti per gli studenti che, in quanto tali,
#non dovrebbero aver bisogno di raccomandazioni.
#I periodici (e tutti i magazine) non devono essere contati dal nostro sistema come libri e vanno dunque filtrati (filtraggio che
#dovrebbe anche consentirci di avere una top k più pulita).
#Per cui, post filtraggio, i generi che rimangono sono le monografie, i manoscritti e (forse) "grafica" e "oggetto". Devo
#controllare cosa si intenda con queste due categorie.
#NOTA: Come risalire ai titoli dei libri da items.csv? NOTA2: per qualche motivo, item_id e manifestation_id sono differenti

file = "/data/SMARTDATA/books/BCT_prestiti/manifestations.csv"
DFmanifestations = spark.read.csv(file,header=True)
DFmanifestations.show()

DFitems.filter(DFitems.manifestation_id == "null").show() #QUESTA RIGA MOSTRA CHE OGNI LIBRO HA IL SUO MANIFESTATION ID

#Il manifestation_id (qualunque cosa sia) è univoco?
DFitems.groupby("manifestation_id").count().agg({"count":"sum"}).show()
DFitems.select("manifestation_id").distinct().groupby("manifestation_id").count().agg({"count":"sum"}).show()


+----------------+----------------+------------+-------------+-------------+--------------------+--------------------+--------------------+-------------------+-------------------+
|manifestation_id|edition_language|edition_date|     ISBNISSN|          EAN|               title|              author|           publisher|     loanable_since|       date_created|
+----------------+----------------+------------+-------------+-------------+--------------------+--------------------+--------------------+-------------------+-------------------+
|          202094|             ita|        2003|   8807920638|         null|Okey dokey, sono ...|   Luciani, Domenica|         Feltrinelli|               null|2003-04-22 00:00:00|
|          910309|             ita|        2017|9788893880442|         null|Che cosa ho in te...|                null|  Baldini & Castoldi|               null|2019-02-20 16:29:27|
|          382413|             ita|        2009|9788860881847|9788860881847|Morte a Firenze :...|   

In [14]:
#MANIFESTATION_ID non è univoco. Devo assolutamente capire su cosa si basa.
#UPDATE: Informandomi nella pagine di FRBR (Functional Requirements for Bibliographic Records) noto che per "manifestation"
#si intende ogni differente versione riferita allo stesso libro in item. Quindi un item può avere più manifestation_id a seconda 
#dell'edizione. Questo vuol dire che, dato che il titolo del libro è solo in manifestations.csv, per avere un titolo riferito
#ad un item_id devo joinare items e manifestations per manifestation_id e poi aggregare item_id con un groupby.

DFmanifestationitems = DFitems.join(DFmanifestations, DFitems.manifestation_id == DFmanifestations.manifestation_id).groupby("item_id", "title").count().show()


+-------+--------------------+-----+
|item_id|               title|count|
+-------+--------------------+-----+
| 176276|L' albero delle p...|    1|
| 511908|Sottovoce o mia N...|    1|
| 831805|I sette in fuga /...|    1|
| 830014|Una vecchia amica...|    1|
| 642490|L'immigrazione / ...|    1|
| 929897|Questa è la mia v...|    1|
|1182043|La nuvola Olga e ...|    1|
| 721676|Giordania : *Amma...|    1|
| 197320|Tempo di fuga : r...|    1|
| 388064|Arrivederci & Ame...|    1|
| 274535|I viaggi di Uliss...|    1|
| 388138|Interni domestici...|    1|
|  71157|Oltre la fine dei...|    1|
| 323888|XY / Sandro Veronesi|    1|
| 839057|2 : *Da Gautama B...|    1|
|2274353|Quack non fa i co...|    1|
| 308043|La Repubblica di ...|    1|
|3044711|Il bambino di Bud...|    1|
| 556793|Il Risorgimento, ...|    1|
|2569140|Caramel / un film...|    1|
+-------+--------------------+-----+
only showing top 20 rows



In [44]:
#Risolto il mistero delle manifestations, procediamo al filtraggio dei generi. Effettuiamo un join fra items e media types per
#avere informazioni sui generi.

DFtypesitems = DFtypes.join(DFitems, DFitems.item_media_id == DFtypes.item_media_id).drop(DFitems.item_media_id)

#Controlliamo che libri sono quelli della categoria "Oggetto" e "Grafica"

DFtypesitems.where((DFtypesitems.item_media_id == "C") | (DFtypesitems.item_media_id == "L")).join(DFmanifestations, DFmanifestations.manifestation_id == DFtypesitems.manifestation_id).groupby("item_id", "title", "item_media_id").count().show()

#A quanto pare le categorie "oggetto" sono quelle basate sui magazine tematici, per cui da scartare; Le categorie "grafica" 
#sono da scartare anch'esse poiché davvero esigue rispetto al catalogo originale.

+-------+--------------------+-------------+-----+
|item_id|               title|item_media_id|count|
+-------+--------------------+-------------+-----+
|3099611|Cucina naturale :...|            L|    1|
|2396909| Le leggi illustrate|            L|    1|
|2525484|      Linux magazine|            L|    1|
|2245941|      Ruoteclassiche|            L|    1|
|2917579|        Motociclismo|            L|    1|
|2756153|Autunno / Ali Smi...|            L|    1|
|2731364|  Le idee di Susanna|            L|    1|
|2942979|Time : the weekly...|            L|    1|
|2912948|Gardenia : fiori,...|            L|    1|
|2415773|Millionaire : lib...|            L|    1|
|2912954|Gardenia : fiori,...|            L|    1|
|2929795|Camminare : per s...|            L|    1|
|2740480|Internazionale : ...|            L|    1|
|2552847|Il mio colore / F...|            C|    1|
|2637404|Sei grandi variaz...|            L|    1|
|2552846|Il mio colore / F...|            C|    1|
|2912955|Gardenia : fiori,...| 

In [45]:
DFfiltered = DFitems.filter((DFitems.item_media_id == "B") | (DFitems.item_media_id == "F"))


+-------+----------------+-------------+---------------+--------------+
|item_id|manifestation_id|item_media_id|home_library_id|inventory_date|
+-------+----------------+-------------+---------------+--------------+
|2555210|          747441|            F|              2|    2016-09-15|
| 218132|            3490|            F|              2|    2008-03-12|
|1009634|          233431|            F|             19|    2004-05-27|
| 121499|          286272|            F|             27|    2010-09-01|
|2400369|          270193|            F|             16|    2016-01-27|
| 103294|          365773|            F|              2|    2010-06-08|
| 164444|          269506|            F|             20|    2006-05-23|
| 188931|          307079|            F|             29|    2006-10-24|
|1310416|          236489|            F|             13|    2004-09-20|
|1283207|           53366|            F|             14|    2006-09-07|
|1191406|           93788|            F|             24|    2004